In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import polars as pl
import nfl_data_py as nfl

env = "local"

In [2]:
if env == "local":
    os.chdir("/Users/samuel/Documents/GitHub/QB-GPT/")
else:
    from google.colab import drive
    drive.mount('/content/gdrive')
    os.chdir("/content/gdrive/MyDrive/NFL_Challenge/NFL-GPT/NFL data")

In [3]:
os.listdir()

['data_models',
 '.DS_Store',
 'app',
 'LICENSE',
 'models',
 'README.md',
 '.gitignore',
 '.gitattributes',
 'data_preprocessing',
 'index',
 '.git',
 'notebooks']

In [4]:
training_data = tf.data.Dataset.load("data_models/Helenos_categ/train_play_prediction_reg")
testing_data = tf.data.Dataset.load("data_models/Helenos_categ/test_play_prediction_reg")

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 5682129699177190627
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 6191096588504482533


In [5]:
train_length = [i for i,_ in enumerate(training_data)][-1] + 1
test_length = [i for i,_ in enumerate(testing_data)][-1] + 1

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 5682129699177190627
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 6191096588504482533


In [6]:
print("Train length is : ", str(train_length))
print("Test length is : ", str(test_length))

Train length is :  290864
Test length is :  124656


In [7]:
batch_size = 32

training_data = training_data.shuffle(train_length).batch(batch_size)
testing_data = testing_data.shuffle(test_length).batch(batch_size)

In [8]:
from models.modeling.QBGPT.models import QBGPT, AttentionBlock

moves_to_pred = 10876
input_size = 10878
starts_size = 1033
scrimmage_size = 100
positions_id = 29
temp_ids = 52

off_def_size = 2
token_type_size = 2
play_type_size = 9



model_tiny = QBGPT(input_vocab_size = input_size,
                    positional_vocab_size = temp_ids,
                    position_vocab_size=positions_id,
                    start_vocab_size=starts_size,
                    scrimmage_vocab_size=scrimmage_size,
                    offdef_vocab_size = off_def_size,
                    type_vocab_size = token_type_size,
                    playtype_vocab_size = play_type_size,
                    embedding_dim = 64,
                    hidden_dim = 64,
                    to_pred_size = moves_to_pred)



model_tiny.load_weights("models/modeling/QBGPT/weights/model_tiny/QBGPT")

In [9]:
class DownEncoder(tf.keras.Model):
  def __init__(self, vocab_size : int, embedding_dim : int):
        super(DownEncoder, self).__init__()

        self.Embedding = tf.keras.layers.Embedding(input_dim = vocab_size,
                                                   output_dim = embedding_dim)

  def call(self, x):
    embed = self.Embedding(x["down_ID"])
    return embed

class SeasonEncoder(tf.keras.Model):
  def __init__(self, vocab_size : int, embedding_dim : int):
        super(SeasonEncoder, self).__init__()

        self.Embedding = tf.keras.layers.Embedding(input_dim = vocab_size,
                                                   output_dim = embedding_dim)

  def call(self, x):
    embed = self.Embedding(x["season_ID"])
    return embed

class TeamEncoder(tf.keras.Model):
  def __init__(self, vocab_size : int, embedding_dim : int):
        super(TeamEncoder, self).__init__()

        self.Embedding = tf.keras.layers.Embedding(input_dim = vocab_size,
                                                   output_dim = embedding_dim)

  def call(self, x):
    embed = self.Embedding(x["team_ID"])
    return embed

class PlayerEncoder(tf.keras.Model):
  def __init__(self, vocab_size : int, embedding_dim : int):
        super(PlayerEncoder, self).__init__()

        self.Embedding = tf.keras.layers.Embedding(input_dim = vocab_size,
                                                   output_dim = embedding_dim)

  def call(self, x):
    embed = self.Embedding(x["player_ids"])
    return embed


class MetaEmbedding(tf.keras.Model):
  def __init__(self, 
               team_vocab_size : int, 
               player_vocab_size : int, 
               season_vocab_size : int, 
               down_vocab_size : int, 
               embedding_dim : int):
        super(MetaEmbedding, self).__init__()

        self.TeamEmbedding = TeamEncoder(vocab_size= team_vocab_size,
                                         embedding_dim=embedding_dim)
        self.PlayerEmbedding = PlayerEncoder(vocab_size= player_vocab_size,
                                             embedding_dim=embedding_dim)
        self.SeasonEmbedding = SeasonEncoder(vocab_size= season_vocab_size,
                                             embedding_dim=embedding_dim)
        self.DownEmbedding = DownEncoder(vocab_size= down_vocab_size,
                                         embedding_dim=embedding_dim)
        
        self.Add = tf.keras.layers.Add()
        
        self.Dense = tf.keras.layers.Dense(embedding_dim, activation = "relu")

  def call(self, x):
    team_embed = self.TeamEmbedding(x)
    
    season_embed = self.SeasonEmbedding(x)
    down_embed = self.DownEmbedding(x)
    
    added = self.Add([team_embed, season_embed, down_embed])
    
    encoded = self.Dense(added)
    
    return encoded

class Helenos(tf.keras.Model):
    def __init__(self, 
                 team_vocab_size : int, 
                 player_vocab_size : int,
                season_vocab_size : int,
                down_vocab_size : int, 
                embedding_dim : int,
                encoder : tf.keras.Model,
                to_pred : int,
                activation : str):
        super(Helenos, self).__init__()
        
        self.Encoder = encoder
        self.MetaEmbedding = MetaEmbedding(team_vocab_size=team_vocab_size,
                                           player_vocab_size = player_vocab_size,
                                           season_vocab_size=season_vocab_size,
                                           down_vocab_size=down_vocab_size,
                                           embedding_dim=embedding_dim)
        
        self.Add = tf.keras.layers.Add()
        
        self.Concat = tf.keras.layers.Concatenate()
        
        self.Attention = AttentionBlock(num_heads=3,
                                        hidden_dim=256,
                                        output_dim=embedding_dim)
        
        self.Dense = tf.keras.layers.Dense(256, activation = "gelu")
        
        self.Pred = tf.keras.layers.Dense(to_pred, activation = activation)
        
    
    def call(self, x):
        encoded_qb_off = self.Encoder(x["off"])
        encoded_meta_off = self.MetaEmbedding(x["off"])
        encoded_off = self.Add([encoded_qb_off, encoded_meta_off])
        
        encoded_qb_def = self.Encoder(x["def"])
        encoded_meta_def = self.MetaEmbedding(x["def"])
        encoded_def = self.Add([encoded_qb_def, encoded_meta_def])
        
        logits_off = self.Attention(encoded_off, x["off"]["pos_ids"], x["off"]["attention_mask"])
        logits_def = self.Attention(encoded_def, x["def"]["pos_ids"], x["def"]["attention_mask"])
        
        logits = self.Concat([logits_off, logits_def])
        
        densed = self.Dense(logits)
        
        preds = self.Pred(densed)
        
        prediction = tf.reduce_mean(preds, axis = -2)
        
        return prediction

In [21]:
model = Helenos(team_vocab_size=32,
                player_vocab_size=21506,
                season_vocab_size= 7,
                down_vocab_size= 5,
                embedding_dim= 64,
                encoder = model_tiny.Encoder,
                to_pred=200,
                activation= "linear")

In [22]:
model.Encoder.trainable = True

In [23]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
                    loss= tf.keras.losses.MeanAbsolutePercentageError(),
                    metrics=[tf.keras.metrics.MeanAbsoluteError(),
                             tf.keras.metrics.MeanSquaredError()])


history_small = model.fit(training_data, validation_data = testing_data, epochs=1)

2023-10-04 00:52:10.369873: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 185271 of 290864


  10/9090 [..............................] - ETA: 2:03 - loss: 39177624.0000 - mean_absolute_error: 5.5607 - mean_squared_error: 87.5429  

2023-10-04 00:52:16.248005: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] Shuffle buffer filled.


9090/9090 [==============================] - 202s 20ms/step - loss: 693961.6250 - mean_absolute_error: 6.2612 - mean_squared_error: 106.0727 - val_loss: 717676.8750 - val_mean_absolute_error: 6.2591 - val_mean_squared_error: 106.1083


In [24]:
preds = []

i = 0

for x,y in testing_data:
    print(i)
    prediction = model.predict(x)
    preds.append(prediction)    
    i+= 1
    if i > 50:
        break

0
1/1 [==============================] - 0s 257ms/step
1
1/1 [==============================] - 0s 29ms/step
2
1/1 [==============================] - 0s 28ms/step
3
1/1 [==============================] - 0s 27ms/step
4
1/1 [==============================] - 0s 29ms/step
5
1/1 [==============================] - 0s 27ms/step
6
1/1 [==============================] - 0s 27ms/step
7
1/1 [==============================] - 0s 27ms/step
8
1/1 [==============================] - 0s 26ms/step
9
1/1 [==============================] - 0s 27ms/step
10
1/1 [==============================] - 0s 27ms/step
11
1/1 [==============================] - 0s 26ms/step
12
1/1 [==============================] - 0s 27ms/step
13
1/1 [==============================] - 0s 28ms/step
14
1/1 [==============================] - 0s 27ms/step
15
1/1 [==============================] - 0s 26ms/step
16
1/1 [==============================] - 0s 25ms/step
17
1/1 [==============================] - 0s 27ms/step
18
1/1 [===========

In [25]:
np.max(np.vstack(preds))

0.049766496

In [26]:
np.min(np.vstack(preds))

-0.039297324

In [27]:
np.mean(np.vstack(preds))

0.00011544645